In [1]:
#run this only once and save the document 
# import urllib.request

# urllib.request.urlretrieve("https://www.treasury.gov/ofac/downloads/ctrylst.txt", "doc.txt")

In [1]:
import pandas as pd
import re
from ast import literal_eval

In [2]:
# function to seperate paragraphs of text into different rows
def per_section(it, is_delimiter=lambda x: x.isspace()):
    ret = []
    for line in it:
        if is_delimiter(line):
            if ret:
                yield ''.join(ret)
                ret = []
        else:
            ret.append(line.rstrip())
    if ret:
        yield ''.join(ret)

with open("doc.txt") as f:
    s = list(per_section(f))
    df1 = pd.DataFrame({'data':s})
    df1.head()

In [3]:
# converting dataframe to extract features
df1 = df1.iloc[3:]
# df1 = df1['data'].drop_duplicates()
# df1 = df1.to_frame()
df1['data'].astype(str)

3        AAFIA SIDDIQUE BRIGADE (a.k.a. JAMAAT-E-AHRAR;...
4        ABDULASATTAR (a.k.a. BARAKZAI, Haji Abdul Satt...
5        ABDULLAH, Abdullah Ahmed (a.k.a. AL-MASRI, Abu...
6        ABDUREHMAN, Ahmed Mohammed (a.k.a. AHMED, Ahme...
7        ABID ALI KHAN TRANSNATIONAL CRIMINAL ORGANIZAT...
                               ...                        
28786    "ZHENG, Gordon" (a.k.a. ZHENG, Fujing; a.k.a. ...
28787    "ZIO" (a.k.a. ZAGARIA, Michele; a.k.a. "CAPAST...
28788    "ZUBEYR, Abu" (a.k.a. ABU ZUBEYR, Muktar Abdir...
28789    "ZUKIPLI" (a.k.a. BIN MARZUKI, Zulkifli; a.k.a...
28790    "ZULKIFLI" (a.k.a. BIN MARZUKI, Zulkifli; a.k....
Name: data, Length: 28788, dtype: object

In [27]:
# import re
# def extract_name(string):
#     name = re.findall(r'^[^\(]+', str(string))[0]
#     string = string.replace(name, '')
#     return name

# def extract_alias(string):
#     aliases_list = re.findall(r'\(.*?\)', string)
#     alias = ''
#     for items in aliases_list:
#         actual_alias = re.findall(r'\(a.k.a', items) 
#         if len(actual_alias)==1:
#             alias = items
# #     string2 = string2.replace(alias, '')
#     return alias

# def extract_dob(string):
#     dob = re.findall(r'DOB.*?\;', string)
#     return dob

# def extract_pob(string):
#     pob = re.findall(r'POB.*?\;', string)
#     return pob

# def extract_tags(string):
#     tags = re.findall(r'\[.*?\]', string)
#     return tags


# df1['name'] = df1['data'].apply(lambda x: extract_name(x))
# df1['alias'] = df1['data'].apply(lambda x: extract_alias(x))
# df1['DOB'] = df1['data'].apply(lambda x: extract_dob(x))
# df1['POB'] = df1['data'].apply(lambda x: extract_pob(x))
# df1['tags'] = df1['data'].apply(lambda x: extract_tags(x))
# string2 = df1.iloc[122]['data']
# print(string2)



In [28]:
# def extract_name(string):
#     name = re.findall(r'^[^\(]+', str(string))[0]
#     string = string.replace(name, '')
#     return name, string

# df1['name'], df1['data1'] = zip(*df1['data'].map(extract_name))

# linked = ''
# linked_list = re.findall(r'\(.*?\)', str(string2))
# for items in linked_list:
#     actual_links = re.findall(r'\(Linked', items) 
#     if len(actual_links)==1:
#         linked += items
# string2 = string2.replace(linked, '')           
# print(linked)

In [4]:
# getting the name, alias name, DOB, POB and tags
def extract_features(string2):
    #finding name and removing it from string 
    name = re.findall(r'^[^\(|;]+', string2)[0]
    string2 = string2.replace(name, '')
    #finding all the aliases
    aliases_list = re.findall(r'\(.*?\)', string2)
    alias = ''
    for items in aliases_list:
        actual_alias = re.findall(r'\(a.k.a', items) 
        if len(actual_alias)==1:
            alias = items
    string2 = string2.replace(alias, '')
    #finding DOB 
    DOB = re.findall(r'DOB.*?\;', string2)
    string2 = string2.replace(';'.join(DOB), '')
    if len(DOB)==0:
        DOB=''
    #finding POB
    POB = re.findall(r'POB.*?\;', string2)
    string2 = string2.replace(';'.join(POB), '')
    if len(POB)==0:
        POB=''
    #finding program
    tags = re.findall(r'\[.*?\]', string2)
    string2 = string2.replace(';'.join(tags), '')
    #organisation links
    linked = ''
    linked_list = re.findall(r'\(.*?\)', str(string2))
    for items in linked_list:
        actual_links = re.findall(r'\(Linked', items) 
        if len(actual_links)==1:
            linked += items
    string2 = string2.replace(linked, '')
    #type
    type_ = re.findall(r'\(individual?\)', string2)
    string2 = string2.replace(str(type_), '')
    if len(type_)==0:
        type_=''
    else: 
        type_= 'individual'
    #nationality
    nationality = re.findall(r'nationality.*?\;', string2)
    string2 = string2.replace(';'.join(nationality), '')
    if len(nationality)==0:
        nationality = ''  
    #address 
    address = string2.split(';')
    #list 
    list_ = 'SDN'
    return name, alias, DOB, POB, tags, address, linked, type_, list_, nationality

In [5]:
df1['name'], df1['alias'], df1['DOB'], df1['POB'], df1['program'], df1['address'], df1['linked orgs'], df1['type'], df1['list'], df1['nationality'] = zip(*df1['data'].map(extract_features))

C:\Users\Dell\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [52]:
df1.nationality.describe()

count     28788
unique      230
top            
freq      22375
Name: nationality, dtype: object

In [14]:
# alias = re.findall(r'\(.*?\)', string)[0]
# new_str = string.replace(alias, '')
df1.loc[:, 'list'] = 'SDN'

C:\Users\Dell\anaconda3\lib\site-packages\pandas\core\indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [39]:
# name = re.findall(r'[A-Z].*?\,|\( ', new_str)[0]
# new_str = new_str.replace(name, '')


df1 = df1.drop('data', 1)
df1.to_csv('terrorist.csv', index=False)

In [221]:
df1['name'] = df1.apply(lambda row : re.findall(r'^[^\(]+', str(row))[0])

In [11]:
df1['type'].unique()

array(['individual', ''], dtype=object)

In [6]:
df1.head()

,data,name,alias,DOB,POB,program,address,linked orgs,type,list,nationality
3,AAFIA SIDDIQUE BRIGADE (a.k.a. JAMAAT-E-AHRAR;...,AAFIA SIDDIQUE BRIGADE,(a.k.a. JAMAAT-E-AHRAR; a.k.a. JAMAATUL AHRAR;...,,,[[SDGT]],"[, Afghanistan, Mohmand Tribal Agency, Pakist...",,,SDN,
4,"ABDULASATTAR (a.k.a. BARAKZAI, Haji Abdul Satt...",ABDULASATTAR,"(a.k.a. BARAKZAI, Haji Abdul Sattar; a.k.a. BA...",[DOB 1964;],"[POB Mirmandaw Village, Nahr-e Saraj District,...",[[SDGT]],"[, KachrayRoad, Pashtunabad, Quetta, Balochist...",(Linked To: HAJI KHAIRULLAH HAJI SATTAR MONEYE...,individual,SDN,
5,"ABDULLAH, Abdullah Ahmed (a.k.a. AL-MASRI, Abu...","ABDULLAH, Abdullah Ahmed","(a.k.a. AL-MASRI, Abu Mohamed; a.k.a. ""ABU MAR...",[DOB 1963;],[POB Egypt;],[[SDGT]],"[, Afghanistan, citizen Egypt (individual).]",,individual,SDN,
6,"ABDUREHMAN, Ahmed Mohammed (a.k.a. AHMED, Ahme...","ABDUREHMAN, Ahmed Mohammed","(a.k.a. AHMED, Ahmed; a.k.a. ALI, Ahmed Mohamm...",[DOB 1965;],[POB Egypt;],[[SDGT]],"[, Afghanistan, citizen Egypt (individual) .]",,individual,SDN,
7,ABID ALI KHAN TRANSNATIONAL CRIMINAL ORGANIZAT...,ABID ALI KHAN TRANSNATIONAL CRIMINAL ORGANIZAT...,,,,[[TCO]],"[, Afghanistan, UnitedArab Emirates .]",,,SDN,
